<a href="https://colab.research.google.com/github/Malanidhruv/quant-projects/blob/main/500_stocks_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install pya3 nsetools quantstats seaborn matplotlib holidays


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 1.3 MB/s eta 0:00:00


In [2]:
pip install quantstats==0.0.50

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.2 MB/s eta 0:00:00
  Attempting uninstall: quantstats
    Found existing installation: QuantStats 0.0.62
    Uninstalling QuantStats-0.0.62:
      Successfully uninstalled QuantStats-0.0.62


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pya3 import Aliceblue
from nsetools import Nse
from datetime import datetime, timedelta
import holidays
import quantstats as qs


In [5]:
import re
import io
import contextlib
import pandas as pd
from datetime import datetime
import quantstats as qs

def get_user_credentials():
    # Replace with actual method to retrieve user credentials
    user_id = '1141826'
    api_key = 'yF7VbKapUqpMvaGv53IMNUK1bSQQiq6VxpW3aCnYHdPCAHWgUEmTA8l8ESoeNCpXLRRqeHhIlycWKAURazd9Pg7WuPTA1V1ks3GATkrNsdGSpzEm59rgkssHOXc5IxLa'
    return user_id, api_key

def parse_metrics(report):
    metrics = {}
    lines = report.split('\n')
    for line in lines:
        match = re.match(r'(.+?)\s+([\d\.\-\%]+)', line)
        if match:
            key, value = match.groups()
            metrics[key.strip()] = value.strip()
    return metrics

def evaluate_stock(metrics, stock_name):
    criteria = {
        'CAGR﹪': 20,           # Compound Annual Growth Rate
        'Sharpe': 1,            # Sharpe Ratio
        'Sortino': 1            # Sortino Ratio
    }

    # Extract relevant metrics
    cagr = float(metrics['CAGR﹪'].strip('%'))
    sharpe = float(metrics['Sharpe'])
    sortino = float(metrics['Sortino'])

    # Evaluate criteria
    if (cagr > criteria['CAGR﹪'] and
        sharpe > criteria['Sharpe'] and
        sortino > criteria['Sortino']):
        return f"Buy {stock_name}"
    else:
        return f"Do Not Buy {stock_name}"

def analyze_stock(alice, stock_token):
    # Define instrument and time range for historical data
    instrument = alice.get_instrument_by_token('NSE', stock_token)
    stock_name = instrument.symbol  # Extract stock name

    from_datetime = datetime(2014, 7, 1)
    to_datetime = datetime(2024, 7, 22)
    interval = "D"

    # Fetch historical data
    historical_data = alice.get_historical(instrument, from_datetime, to_datetime, interval)

    # Create DataFrame
    df = pd.DataFrame(historical_data)

    # Preprocess data
    df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%d %H:%M:%S')
    df.set_index('datetime', inplace=True)
    df['pct'] = df['close'].pct_change()
    df.columns = df.columns.str.strip()
    df = df.drop(columns=['open', 'high', 'low', 'close', 'volume']).dropna()

    # Capture the metrics report as a string
    buffer = io.StringIO()
    with contextlib.redirect_stdout(buffer):
        qs.reports.metrics(mode='basic|full', returns=df['pct'])
    report = buffer.getvalue()

    # Parse the metrics from the report
    metrics = parse_metrics(report)

    # Make a buy recommendation based on metrics
    recommendation = evaluate_stock(metrics, stock_name)
    return recommendation

def main():
    # Load CSV file
    file_path = '/content/NIFTY 500 stocks.csv'
    stocks_df = pd.read_csv(file_path)

    # Get user credentials
    user_id, api_key = get_user_credentials()

    # Create an instance of Aliceblue
    alice = Aliceblue(user_id=user_id, api_key=api_key)

    # Get session ID
    alice.get_session_id()

    # Iterate over each stock token
    buy_recommendations = []
    for _, row in stocks_df.iterrows():
        stock_token = int(row['Symbol'])  # Adjust column name if necessary
        try:
            recommendation = analyze_stock(alice, stock_token)
            if recommendation.startswith("Buy"):
                buy_recommendations.append((stock_token, recommendation))
        except Exception as e:
            print(f"Error analyzing stock {stock_token}: {e}")

    # Print buy recommendations
    for stock_token, recommendation in buy_recommendations:
        print(f"Stock {stock_token}: {recommendation}")

if __name__ == "__main__":
    main()


NOTE: Today's contract master file will be updated after 08:00 AM. Before 08:00 AM previous day contract file be downloaded.
Stock 3432: Buy TATACONSUM
Stock 2955: Buy KALYANKJIL
Stock 18060: Buy CONCORDBIO
Stock 19020: Buy JSWINFRA
Stock 18721: Buy NUVAMA
Stock 14154: Buy UNOMINDA
Stock 12092: Buy KAYNES
Stock 958: Buy ESCORTS
Stock 6994: Buy BSOFT
Stock 7242: Buy METROBRAND
Stock 21174: Buy CDSL
Stock 20551: Buy DOMS
Stock 13310: Buy KEI
Stock 10217: Buy ADANIENSOL
Stock 18457: Buy POWERINDIA
Stock 19813: Buy HONASA
Stock 19913: Buy DMART
Stock 2664: Buy PIDILITIND
Stock 11966: Buy BIKAJI
Stock 3563: Buy ADANIGREEN
Stock 22377: Buy MAXHEALTH
Stock 18566: Buy RRKABEL
Stock 3563: Buy ADANIGREEN
Stock 1949: Buy KSB
Stock 7145: Buy ANANDRATHI
Stock 2854: Buy CRAFTSMAN
Stock 7929: Buy ZYDUSLIFE
Stock 12489: Buy SWSOLAR
Stock 2854: Buy CRAFTSMAN
Stock 14180: Buy NSLNISP
Stock 13359: Buy KFINTECH
Stock 342: Buy CAMS
Stock 19943: Buy DEEPAKNTR
Stock 17903: Buy ABBOTINDIA
Stock 16915: Buy ZFC